<a href="https://colab.research.google.com/github/shazzad-hasan/Training-Reproducable-Deep-Learning-Models/blob/main/LeNet-5/lenet5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/shazzad-hasan/Training-Reproducable-Deep-Learning-Models.git

Cloning into 'Training-Reproducable-Deep-Learning-Models'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 13 (delta 2), reused 4 (delta 0), pack-reused 0
Unpacking objects: 100% (13/13), done.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/Training-Reproducable-Deep-Learning-Models/LeNet-5

/content/Training-Reproducable-Deep-Learning-Models/LeNet-5


In [4]:
# import required libraries
import torch
import torchvision

import numpy as np
import matplotlib.pyplot as plt

In [5]:
# import local helper functions
from helper_train import train
from helper_evaluate import compute_loss_accuracy
from helper_plot import show_sample_test_result

In [6]:
# check if cuda is available
train_on_gpu = torch.cuda.is_available()

if train_on_gpu:
  print("CUDA is available!")
else:
  print("CUDA is not available")

if train_on_gpu:
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

CUDA is available!
